In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

print("Check!")

Check!


In [2]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )
print("Check!")

Check!


from pydrake.all import (
    autoDiffToGradientMatrix, initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix)

**These imports seem to be renovated, should have equivalent. Do check!** 

In [3]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [4]:
table_dir = ("/Users/xiao/0_codes/usingDrake/models/objects")
block_dir = ("/Users/xiao/0_codes/usingDrake/models/ycb/sdf/")
table_file = os.path.join(table_dir, "table_top.sdf")
block_file = os.path.join(block_dir, "003_cracker_box.sdf")

# add table 
table = parser.AddModels(table_file)
box = parser.AddModels(block_file)

"""
# This is explicitly fixing the table to world frame. What happens if it is not
# explicitly fixed here but "not publishing to update its state" ?? See later sections.  

# fix the centre of table to world
table_body_frame = plant.GetFrameByName("table_top_center")
plant.WeldFrames(plant.world_frame(), table_body_frame) 
"""

'\n# This is explicitly fixing the table to world frame. What happens if it is not\n# explicitly fixed here but "not publishing to update its state" ?? See later sections.  \n\n# fix the centre of table to world\ntable_body_frame = plant.GetFrameByName("table_top_center")\nplant.WeldFrames(plant.world_frame(), table_body_frame) \n'

In [5]:
plant.Finalize()
plant.set_name("table_with_box")

In [6]:
# get free body box's frame and its id in the plant
box_body_frame = plant.GetBodyByName("base_link_cracker")
box_body_id = plant.GetBodyFrameIdIfExists(box_body_frame.index())
print(f"box_body_frame is {box_body_frame} \n")
print(f"box_body_id is {box_body_id}")

box_body_frame is <RigidBody name='base_link_cracker' index=2 model_instance=3> 

box_body_id is <FrameId value=23>


In [7]:
draw_frames = True
frames_to_draw = [box_body_id] if draw_frames else []

In [8]:
# now build the diagram
diagram = builder.Build()

# create the default diagram's context 
context = diagram.CreateDefaultContext()

# get the mutable context for updates
mutable_context = plant.GetMyContextFromRoot(context) # seems equivalent to .GetMyMutableContextFromRoot()

# move the table for a bit
X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])
plant.SetFreeBodyPose(mutable_context, plant.GetBodyByName("table_top_link"), X_W_table)

# move the box a bit relative to the table-top
X_table_box = RigidTransform(RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.1, -0.2, 0.])

# get table-top frame in the world, here the dummy is for getting familiar with Drake
table_top_frame = plant.GetFrameByName("table_top_center")
X_W_table_dummy = table_top_frame.CalcPoseInWorld(mutable_context)
X_W_box = X_W_table_dummy.multiply(X_table_box)
plant.SetFreeBodyPose(mutable_context, box_body_frame, X_W_box)

In [10]:
meshcat = StartMeshcat()
# not yet publish any context for visualization

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a, b):
    return np.array_equal(a, b) and np.array_equal(autoDiff